In [1]:
import subprocess
import json
import os
from constants import BTC, BTCTEST, ETH
from dotenv import load_dotenv
from bip_utils import Bip32, Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
from web3.auto.gethdev import w3

In [2]:
# Load and set environment variables
load_dotenv('mnemonic.env')
mnemonic= os.getenv("mnemonic")

In [3]:
print(BTCTEST)
#print(mnemonic)

btc-test


In [4]:
#Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --coin="{coin}" --numderive="{numderive}" --cols=path,address,privkey,pubkey --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

numderive = 3

# Create a dictionary object called coins to store the output from `derive_wallets`.

coins = {"btc-test": derive_wallets(mnemonic, BTCTEST, numderive), "eth": derive_wallets(mnemonic, ETH, numderive)}


In [5]:
coins

{'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'mqwUTRp6LmdbdzDK9wR79bGWY4ccrEcfV5',
   'privkey': 'cTkg7fHbvED8eacqCUsAkUJmSJv7LWjbqDZcv2t2LwdNyZL2muRa',
   'pubkey': '026e74912bb41ea3132b57369531a845f538db41575148bfd75fb410cbb1480237'},
  {'path': "m/44'/1'/0'/0/1",
   'address': 'mifowmkNiwYm3Dxo2tABZiQU2yy67FEDJS',
   'privkey': 'cQ4JD4dL2PVW5YbJypRg7ZjxLNouKnxjFPqknH13rxLmQoHpq83o',
   'pubkey': '029f83387003bc5ea1766e63842bb44d1235196abe5b9ddab09ad8e90adfc2b05f'},
  {'path': "m/44'/1'/0'/0/2",
   'address': 'moDKqkjmr97WxLtHaHimTTP4HceCx25qfD',
   'privkey': 'cQoxfK9cPEgkCdwf8AHPseezAdgzzyWuj5UJ7dWm9nCEFJPPRDSQ',
   'pubkey': '0272b4122f6cee823386c8ac44e9d0492d6c7a7354f796a0e211771387ce1c184b'}],
 'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0x2Ff466B4824B924D99ba079fa3f76FFf5eC53a7B',
   'privkey': '0x3619968a4f0f2c1d7b46e44bb434844aaf8b772504a278d1059815b2d67ef26b',
   'pubkey': '027796808fb9e92116399cf82e1c08a6e32db1adab7ac6568fd96b419c65f796bb'},
  {'path': "m

In [6]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, receiver, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to": to, "value": amount}
        )
        return {
            "from": eth_acc.address,
            "to": receiver,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address),
            "chainID": web3.eth.chain_id,
        }

    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address,[(receiver, amount, BTC)])

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, receiver, amount):
    txn = create_tx(coin, account, receiver, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        return result
    
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, receiver, amount)
        signed_txn = account.sign_transaction(txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [7]:
btc_sender = coins["btc-test"][0]['privkey']
#sending account
btc_receiver = coins["btc-test"][1]['address']
#receiving account

#send_tx(BTCTEST, priv_key_to_account(btc-test, btc_key),btc_address, 0.00000)

In [8]:
btc_sender

'cTkg7fHbvED8eacqCUsAkUJmSJv7LWjbqDZcv2t2LwdNyZL2muRa'

In [9]:
btc_receiver

'mifowmkNiwYm3Dxo2tABZiQU2yy67FEDJS'

In [10]:
#create_tx(BTCTEST,priv_key_to_account(BTCTEST, btc_sender) ,btc_receiver, 0.00001)

In [11]:
#send_tx(BTCTEST,priv_key_to_account(BTCTEST, btc_sender) ,btc_receiver, 0.00001)